In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os 

import sys; sys.path.append('/home/mitch/util/python')
sys.path.append('/home/mitch/school/mfl/src/')
import utils

save = False

In [95]:
raw = '/home/mitch/Dropbox/data/mexico_fls/raw/2005/'
interim = '/home/mitch/Dropbox/data/mexico_fls/interim/2005/'

figs = '/home/mitch/school/mfl/fig/'
dicts= '/home/mitch/school/mfl/notebooks/dicts/'

In [96]:
os.chdir(dicts)
import json_utils

education = json_utils.load_json('education.json')

In [97]:
ids = ['folio', 'ls', 'pid_link']
keep = ['folio', 'ls', 'pid_link']
os.chdir(raw + 'book IIIB/data/')
portad = pd.read_stata('iiib_portad.dta')
for id in ['folio', 'ls', 'pid_link']:
    portad[id] = portad[id].astype('int64')
portad = portad.sort_values(by=['folio', 'ls'])

In [98]:
os.chdir(raw + 'book IIIA/data/')
weights_3a = pd.read_stata('weights.dta')
for id in ['folio', 'ls']:
    weights_3a[id] = weights_3a[id].astype('int64')
weights_3a = (weights_3a.query('fac_3a>0')
              .drop_duplicates()
              .sort_values(by=['folio', 'ls']))

weights_3a = weights_3a.rename(columns={'fac_3a':'weight_ind'})

keep = keep + ['weight_ind']


In [99]:
# ids
os.chdir(raw + 'book C/data/')
data_ls = pd.read_stata('c_ls.dta')
data_ls[['folio', 'ls']] = data_ls[['folio', 'ls']].astype('int64')
data_ls = data_ls.drop(columns='pid_link')

os.chdir(raw + 'book IIIA/data/')
# employment
data_tb = pd.read_stata('iiia_tb.dta')
data_tb[ids] = data_tb[ids].astype('int64')
data_tb = data_tb.drop(columns='pid_link')

# education
data_ed = pd.read_stata('iiia_ed.dta')
data_ed[ids] = data_ed[ids].astype('int64')
data_ed = data_ed.drop(columns='pid_link')

# non labor income, is ANNUAL
data_iin = pd.read_stata('iiia_iin.dta')
data_iin[ids] = data_iin[ids].astype('int64')
data_iin = data_iin.drop(columns='pid_link')


os.chdir(raw + 'book IIIB/data/')
# credit
data_cr = pd.read_stata('iiib_cr.dta')
data_cr[ids] = data_cr[ids].astype('int64')
data_cr = data_cr.drop(columns='pid_link')

# credit 1
data_cr1 = pd.read_stata('iiib_cr1.dta')
data_cr1[ids] = data_cr1[ids].astype('int64')
data_cr1 = data_cr1.drop(columns='pid_link')

# family structure (gender)
data_thi = pd.read_stata('iiib_thi.dta')
data_thi[ids] = data_thi[ids].astype('int64')
data_thi = data_thi.drop(columns='pid_link')

In [100]:
data_ls[['folio', 'ls']].drop_duplicates()

,folio,ls
0,1000,1
1,1000,2
2,1000,3
3,1000,4
4,1000,5
...,...,...
38218,10758000,2
38219,10758000,3
38220,10759000,1
38221,10759000,2


In [101]:
data = (portad.merge(weights_3a, on=['folio', 'ls'], how='outer')
        .merge(data_ls, on=['folio', 'ls'], how='outer')
        .merge(data_tb, on=['folio', 'ls'], how='outer')
        .merge(data_ed, on=['folio', 'ls'], how='outer')
        .merge(data_cr, on=['folio', 'ls'], how='outer')
        .merge(data_cr1, on=['folio', 'ls'], how='outer')
        .merge(data_thi, on=['folio', 'ls'], how='outer')
        )
data = data.copy()

In [102]:
data.sort_values(by=['folio', 'ls']).query('edad <= 65').query('edad >= 20')

,catorcena,edad,edo_civil,ent,folio,ls,pid_link,rel,weight_ind,ls00,...,thi03_1,thi03_2,thi04,thi05_1,thi05_2,thi15,thi16_1,thi16_2,thi17_1,thi17_2
0,3.0,41.0,3.0,20.0,1000,1,1.000010e+05,20.0,2244.0,01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,40.0,3.0,20.0,1000,2,1.000020e+05,20.0,2244.0,02,...,NaN,NaN,1.0,3.0,NaN,1.0,NaN,NaN,1.0,1.0
2,3.0,20.0,6.0,20.0,1003,1,1.003010e+05,20.0,NaN,01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,45.0,3.0,20.0,2000,2,2.000020e+05,20.0,2244.0,02,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3.0,22.0,6.0,20.0,2000,3,2.000030e+05,20.0,2244.0,03,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21052,10.0,64.0,3.0,19.0,10757000,2,1.075700e+09,20.0,839.0,02,...,NaN,NaN,1.0,1.0,3.0,1.0,NaN,NaN,1.0,5.0
21053,99.0,27.0,3.0,19.0,10758000,1,1.075800e+09,20.0,839.0,01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21054,11.0,24.0,3.0,19.0,10758000,2,1.075800e+09,20.0,839.0,02,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21055,99.0,25.0,3.0,19.0,10759000,1,1.075900e+09,20.0,839.0,01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
data['age'] = data['edad']
data['state'] = data['ent']
hh_size = data.groupby('folio')['ls'].size().reset_index().rename(columns={'ls':'hh_size'})
data = data.merge(hh_size, how='outer', on=['folio'])
keep = keep + ['age', 'state', 'hh_size']


In [104]:
# drop low / dk values of education
data['education'] = data['ed06']
data.loc[data.education.isna(), 'education'] = 98.0
todrop = [98.0] 
keeped = [ed not in todrop for ed in data['education']]
data = data.loc[keeped]
data['education_old'] = data['education'].copy()
data['education'] = data['education'].astype(str).apply(lambda x : education[x])
keep = keep + ['education']

In [105]:
data['gender'] = data['thi01'].apply(utils.make_gender)
keep = keep + ['gender']

In [106]:
# Examining Income

# value to substitute if worker doesn't know wage
# set to np.nan to implicitly drop from observations

dk_value = 0
data['total_annual_income'] = 0

data['annual_income'] = 0
data.loc[data['tb36a_1'] == 3.0, 'annual_income'] = data.loc[data['tb36a_1'] ==3.0, 'tb36a_2']
# implicitly drop people that do not know their income level
data.loc[data['tb36a_1'] == 8.0, 'annual_income'] = np.nan

data['annual_wage'] = 0
earns_wage = data['tb36aa_1'] == 'A'
data.loc[earns_wage, 'annual_wage'] = data.loc[earns_wage, 'tb36aa_2']
data.loc[data['tb36aa_1'] == '8', 'annual_wage'] = dk_value

data['annual_piecework'] = 0
earns_piecework = data['tb36ab_1'] == 'B'
data.loc[earns_piecework, 'annual_piecework'] = data.loc[earns_piecework, 'tb36ab_2']
data.loc[data['tb36ab_1'] == '8', 'annual_piecework'] = dk_value

data['annual_tips'] = 0
earns_tips = data['tb36ac_1'] == 'C'
data.loc[earns_tips, 'annual_tips'] = data.loc[earns_tips, 'tb36ac_2']
data.loc[data['tb36ac_1'] == '8', 'annual_tips'] = dk_value

data['annual_extrahours'] = 0
earns_extrahours = data['tb36ad_1'] == 'D'
data.loc[earns_extrahours, 'annual_extrahours'] = data.loc[earns_extrahours, 'tb36ad_2']
data.loc[data['tb36ad_1'] == '8', 'annual_extrahours'] = dk_value

data['annual_christmasbonus'] = 0
earns_christmasbonus = data['tb36ae_1'] == 'E'
data.loc[earns_christmasbonus, 'annual_christmasbonus'] = data.loc[earns_christmasbonus, 'tb36ae_2']
data.loc[data['tb36ae_1'] == '8', 'annual_christmasbonus'] = dk_value

data['annual_bonus'] = 0
earns_bonus = data['tb36af_1'] == 'F'
data.loc[earns_bonus, 'annual_bonus'] = data.loc[earns_bonus, 'tb36af_2']
data.loc[data['tb36af_1'] == '8', 'annual_bonus'] = dk_value

data['annual_profitdistribution'] = 0
earns_profitdistribution = data['tb36ah_1'] == 'H'
data.loc[earns_profitdistribution, 'annual_profitdistribution'] = data.loc[earns_profitdistribution, 'tb36ah_2']
data.loc[data['tb36ah_1'] == '8', 'annual_profitdistribution'] = dk_value

data['annual_other'] = 0
earns_other = data['tb36am_1'] == 'M'
data.loc[earns_other, 'annual_other'] = data.loc[earns_other, 'tb36am_21']
data.loc[data['tb36am_1'] == '8', 'annual_other'] = dk_value


hasSecondJob = data['tb36b_1'] == 3.0
data['annual_income_secondjob'] = 0.0
data.loc[hasSecondJob, 'annual_income_second_job'] = data.loc[hasSecondJob, 'tb36b_2']

hasMainBusiness = data['tb38p2_1'] == 2.0
data['netIncomeMainBusiness'] = 0.0
data.loc[hasMainBusiness, 'netIncomeMainBusiness'] = data.loc[hasMainBusiness, 'tb38p2_2']

hasSecondBusiness = data['tb38s2_1'] == 2.0
data['netIncomeSecondBusiness'] = 0.0
data.loc[hasSecondBusiness, 'netIncomeSecondBusiness'] = data.loc[hasSecondBusiness, 'tb38s2_2']


income_sources = [
                  'annual_income',
                  'annual_wage',
                  'annual_piecework',
                  'annual_tips',
                  'annual_extrahours',
                  'annual_other',
                  'annual_christmasbonus',
                  'annual_bonus',
                  'annual_profitdistribution',
                  'annual_income_second_job'
                  ]
                  #'netIncomeMainBusiness', 'netIncomeSecondBusiness']

data['annual_labor_income'] = data[income_sources].sum(axis=1)

In [107]:
data

,catorcena,edad,edo_civil,ent,folio,ls,pid_link,rel,weight_ind,ls00,...,annual_extrahours,annual_christmasbonus,annual_bonus,annual_profitdistribution,annual_other,annual_income_secondjob,annual_income_second_job,netIncomeMainBusiness,netIncomeSecondBusiness,annual_labor_income
0,3.0,41.0,3.0,20.0,1000,1,100001.0,20.0,2244.0,01,...,0,0,0,0,0,0.0,NaN,0.0,0.0,0.0
1,3.0,40.0,3.0,20.0,1000,2,100002.0,20.0,2244.0,02,...,0,0,0,0,0,0.0,NaN,0.0,0.0,0.0
5,3.0,20.0,6.0,20.0,1003,1,100301.0,20.0,NaN,01,...,5000,0,0,0,0,0.0,NaN,0.0,0.0,8000.0
6,3.0,19.0,6.0,20.0,1003,2,100003.0,20.0,NaN,02,...,0,0,0,0,0,0.0,NaN,0.0,0.0,0.0
7,3.0,45.0,3.0,20.0,2000,2,200002.0,20.0,2244.0,02,...,0,0,0,0,0,0.0,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37945,NaN,NaN,NaN,NaN,8556000,2,NaN,NaN,5284.0,02,...,0,0,0,0,0,0.0,NaN,0.0,0.0,0.0
37953,NaN,NaN,NaN,NaN,9271000,2,NaN,NaN,4611.0,02,...,0,0,0,0,0,0.0,NaN,0.0,0.0,0.0
37956,NaN,NaN,NaN,NaN,9959000,2,NaN,NaN,1401.0,02,...,0,0,0,0,0,0.0,NaN,0.0,0.0,0.0
38087,NaN,NaN,NaN,NaN,1801003,2,NaN,NaN,NaN,02,...,0,0,0,0,0,0.0,NaN,0.0,0.0,0.0


# Debts

In [108]:
# more formal measure of debts
data['asked_for_loan_12mth'] = data['cr10'] == 1.0
data['not_asked_for_loan_32mth'] = data['cr10'] == 1.0

loan_locations = {
    1.0:'bank',
    2.0:'savings_fund',
    3.0:'moneylender',
    4.0:'relative',
    5.0:'friends',
    6.0:'work',
    7.0:'pawnshop',
    8.0:'verbal_agreement_credit_program',
    9.0:'other_govt',
    10.0:'other',
    -1.0: np.nan
}
formal = ['bank', 'savings_fund', 'moneylender', 'pawnshop']
informal = ['relative', 'friends', 'work', 'verbal_agreement_credit_program', 'other_govt', 'other']
friends_relatives = ['relative', 'friends']

data.loc[np.isnan(data['cr14_1']), 'cr14_1'] = -1.0


In [109]:
data['where_get_loan'] = data['cr14_1'].apply(lambda x        : loan_locations[x])
data['has_loan_formal']    = data['where_get_loan'].apply(lambda x: x in formal)
data['has_loan_informal']  = data['where_get_loan'].apply(lambda x: x in informal)
data['has_loan_friends_relatives']  = data['where_get_loan'].apply(lambda x: x in friends_relatives)
data['has_loan'] = data['has_loan_formal'] | data['has_loan_informal'] | data['has_loan_friends_relatives']

got_loan = data['cr16'] == 1.0

data['loan_size'] = 0.0
data.loc[got_loan, 'loan_size'] = data.loc[got_loan, 'cr19']

data['loan_formal']   = data['loan_size'] * (data['has_loan_formal'] + 0.0)
data['loan_informal'] = data['loan_size'] * (data['has_loan_informal'] + 0.0)
data['loan_friends_relatives']   = data['loan_size'] * (data['has_loan_friends_relatives'] + 0.0)


# Savings

In [110]:
data['has_savings'] = data['cr27'] == 1.0
data['no_savings'] = data['cr27'] == 3.0
dk_savings = data['cr27'] == 8.0

data['savings'] = np.nan
data.loc[data.has_savings, 'savings'] = data.loc[data.has_savings, 'cr28']
data.loc[data.no_savings, 'savings'] = 0.0


In [111]:
savings_no_savings     = data['cr29_1a'] == 1.0
savings_bank           = data['cr29_1b'] == 2.0
savings_coop           = data['cr29_1c'] == 3.0
savings_savings_bank   = data['cr29_1d'] == 4.0
savings_friend_not_hhm = data['cr29_1e'] == 5.0
savings_afores         = data['cr29_1f'] == 6.0
savings_caja_solidaria = data['cr29_1g'] == 7.0
savings_house          = data['cr29_1h'] == 8.0
savings_work           = data['cr29_1i'] == 9.0
savings_other          = data['cr29_1j'] == 10.0

In [112]:
data['has_savings_formal']   = savings_bank | savings_coop | savings_savings_bank | savings_caja_solidaria
data['has_savings_informal'] = savings_friend_not_hhm | savings_house | savings_work 

In [113]:
data['savings_formal']   = (data['has_savings_formal'] + 0.0) * data['cr28']
data['savings_informal'] = (data['has_savings_informal'] + 0.0) * data['cr28']

data.loc[data.no_savings, 'savings_formal'] = 0
data.loc[data.no_savings, 'savings_informal'] = 0

both = data['has_savings_formal'] & data['has_savings_informal']

data.loc[both, 'savings_formal']   = np.nan
data.loc[both, 'savings_informal'] = np.nan

In [114]:
data['has_liquid_formal'] = data['has_savings_formal'] | data['loan_formal']
data['liquid_formal'] = data['savings_formal'] - data['loan_formal']

In [115]:
data['has_liquid'] = data['has_savings'] | data['has_loan']

In [116]:
import importlib
importlib.reload(utils)
data = utils.sum_over_household(data, 'savings')
data = utils.sum_over_household(data, 'savings_formal')
data = utils.sum_over_household(data, 'savings_informal')
data = utils.sum_over_household(data, 'annual_labor_income')
data = utils.sum_over_household(data, 'annual_wage')
data = utils.sum_over_household(data, 'annual_piecework')
data = utils.sum_over_household(data, 'annual_tips')
data = utils.sum_over_household(data, 'annual_extrahours')
data = utils.sum_over_household(data, 'annual_other')
data = utils.sum_over_household(data, 'annual_christmasbonus')
data = utils.sum_over_household(data, 'annual_bonus')
data = utils.sum_over_household(data, 'annual_profitdistribution')
data = utils.sum_over_household(data, 'annual_income_second_job')
data = utils.sum_over_household(data, 'netIncomeMainBusiness')
data = utils.sum_over_household(data, 'netIncomeSecondBusiness')
data = utils.sum_over_household(data, 'loan_formal')
data = utils.sum_over_household(data, 'loan_informal')
data = utils.sum_over_household(data, 'loan_friends_relatives')
data = utils.sum_over_household(data, 'liquid_formal')


data = utils.any_in_household(data, 'has_liquid')
data = utils.any_in_household(data, 'has_savings')
data = utils.any_in_household(data, 'has_savings_formal')
data = utils.any_in_household(data, 'has_savings_informal')
data = utils.any_in_household(data, 'has_loan_formal')
data = utils.any_in_household(data, 'has_loan_informal')
data = utils.any_in_household(data, 'has_loan_friends_relatives')
data = utils.any_in_household(data, 'has_loan')
data = utils.any_in_household(data, 'has_liquid_formal')
data = utils.any_in_household(data, 'asked_for_loan_12mth')


In [117]:
keep += ['has_liquid_hh', 'savings_hh', 'savings_formal_hh', 'savings_informal_hh']
keep += ['has_savings_hh', 'has_savings_formal_hh', 'has_savings_informal_hh']
keep += ['loan_formal_hh', 'loan_informal_hh', 'loan_friends_relatives_hh']
keep += ['has_loan_hh', 'has_loan_formal_hh', 'has_loan_informal_hh', 'has_loan_friends_relatives_hh']
keep += ['liquid_formal_hh', 'has_liquid_formal_hh']
keep += ['annual_labor_income_hh', 'netIncomeMainBusiness_hh', 'netIncomeSecondBusiness_hh']

In [118]:
inner = data[['folio', 'annual_labor_income']].groupby('folio')['annual_labor_income'].apply(np.sum).rename('annual_labor_income_hh_inner')
outer = data[['folio', 'annual_labor_income']].groupby('folio')['annual_labor_income'].apply(np.sum).rename('annual_labor_income_hh_outer')
data_inner = data.copy().merge(inner, on='folio', how='inner')
data_outer = data.copy().merge(outer, on='folio', how='outer')

claim: nothing wrong with my current methodology, but I am implicitly dropping those other measures

barely any lost due to later ones
let's focus on annual income

according to this most of these are singleton households

# loans regardless of location

In [119]:
data['cr14_1'].unique()
data['asked_for_loan'] = data['cr14_1'] != -1

In [120]:
keep

['folio',
 'ls',
 'pid_link',
 'weight_ind',
 'age',
 'state',
 'hh_size',
 'education',
 'gender',
 'has_liquid_hh',
 'savings_hh',
 'savings_formal_hh',
 'savings_informal_hh',
 'has_savings_hh',
 'has_savings_formal_hh',
 'has_savings_informal_hh',
 'loan_formal_hh',
 'loan_informal_hh',
 'loan_friends_relatives_hh',
 'has_loan_hh',
 'has_loan_formal_hh',
 'has_loan_informal_hh',
 'has_loan_friends_relatives_hh',
 'liquid_formal_hh',
 'has_liquid_formal_hh',
 'annual_labor_income_hh',
 'netIncomeMainBusiness_hh',
 'netIncomeSecondBusiness_hh']

In [121]:
data['is_hhm'] = data['ls05_1'] 
data = data[data['is_hhm'] == 1.0]

save = True
os.chdir(interim)
if save:
    data[keep].to_stata('ind_2005.dta', write_index=False)
    print('saved')


saved


In [33]:
data['ls05_1'].value_counts()

ls05_1
3.0     6090
1.0     5952
2.0     5125
5.0      644
10.0     358
8.0      208
6.0      116
13.0     113
4.0       99
9.0       75
7.0       58
17.0      18
18.0      15
14.0      14
12.0      12
15.0       5
11.0       4
Name: count, dtype: int64

In [ ]:
data_loans = data.copy()
data_loans = data_loans[~np.isnan(data['cr23_1'])]

keep_loans = ['weight_ind']
keep_loans += ['asked_for_loan_12mth', 'where_get_loan', 'loan_size']

rename_loans = {
    'cr21_1':'time_to_pay_loan',
    'cr21_21':'years_to_pay_loan',
    'cr21_22':'months_to_pay_loan',
    'cr21_23':'days_to_pay_loan'
    }
data_loans = data_loans.rename(columns = rename_loans)
keep_loans += ['time_to_pay_loan', 'years_to_pay_loan', 'months_to_pay_loan', 'days_to_pay_loan']

interest_rate_type = {
    1.0   : 'annual',
    2.0   : 'monthly',
    3.0   : 'daily',
    4.0   : 'no_interest',
    8.0   : 'DK',
}

rename_interest_rate = {
    'cr23_2':'annual_interest_rate',
    'cr23_3':'monthly_interest_rate',
    'cr23_4':'daily_interest_rate'
}
data_loans = data_loans.rename(columns = rename_interest_rate)
keep_loans += ['annual_interest_rate', 'monthly_interest_rate', 'daily_interest_rate']

data_loans['interest_rate_type'] = data_loans['cr23_1'].apply(lambda x : interest_rate_type[x])
keep_loans += ['interest_rate_type']

os.chdir(interim)
if save:
    data_loans[keep_loans].to_stata('loans_2005.dta', write_index=False)
    print('saved')

